<a href="https://colab.research.google.com/github/rohitratannagar/AgentsByRohit/blob/main/StoryTellerAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📖 Story Teller Agent

Story Teller Agent is an AI-powered storytelling system that generates creative stories from user prompts and converts them into Audio narration.

### ✨ Features
- 🧠 AI-generated stories
- 🔊 can play story
- 💾 Automatic story saving for replay
- 🖇 can access all previous generated stories

Turn your ideas into spoken stories — instantly. 🎧✨

In [ ]:
from openai import OpenAI
from google.colab import userdata

In [ ]:
gemini_api_key = userdata.get('GEMINI_API_KEY2')
print(f"gemini api key starts with {gemini_api_key[:5]}")

In [63]:
import json
from IPython.display import Audio, Markdown


In [ ]:
Audio(f"/content/sample_data/audio.wav")

In [ ]:
system_prompt = f"""You are an audio-first storyteller.

Your task is to write stories that are meant to be spoken aloud by a text-to-speech engine.

Follow these rules strictly:
- Write in short, clear sentences.
- Use natural pauses by adding extra spaces or line breaks.
- Avoid complex or long sentences.
- Use simple vocabulary that is easy to pronounce.
- Add emotion using punctuation, not formatting.
- Insert pauses using ellipses (...) and double spaces.
- Do NOT use emojis, markdown, or special symbols.
- Do NOT use dialogue tags like “he said loudly”.
- Let pauses and sentence structure create emotion.
- Write in a calm, expressive, human-like tone.

The output must sound natural when read aloud by an AI voice.

Now write the story."""

In [81]:
import pyttsx3
import os

def save_story(
    name: str,
    content: str,
    path: str = "/content/sample_data",
    rate: int = 140
):
    os.makedirs(path, exist_ok=True)
    file_path = os.path.join(path, f"{name}.wav")

    engine = pyttsx3.init()

    # ---- Storytelling voice settings ----
    engine.setProperty("rate", rate)     # slower = clearer narration
    engine.setProperty("volume", 1.0)    # max clarity

    voices = engine.getProperty("voices")

    # Prefer female / natural voice if available
    for v in voices:
        if "female" in v.name.lower() or "zira" in v.name.lower():
            engine.setProperty("voice", v.id)
            break
    else:
        engine.setProperty("voice", voices[0].id)

    # ---- Save narration ----
    engine.save_to_file(content, file_path)
    engine.runAndWait()

    return file_path

In [84]:
from IPython.display import Audio

def play_story(file_name):
  # Return an IPython.display.Audio object
  return f"/content/sample_data/{file_name}.wav"


In [85]:
def get_all_stories():
  stories = []
  for file in os.listdir("/content/sample_data"):
    if file.endswith(".wav"):
      stories.append(file)

  return stories

In [86]:
get_all_stories()

['arrogant_lion.wav',
 'arrogant_queen.wav',
 'audio.wav',
 'spoiled_boy.wav',
 'horror_story.wav',
 'arrogant_boy.wav']

In [67]:
save_story_json = {
  "name": "save_story",
  "description": "Convert a story text into a WAV audio file using text-to-speech and save it locally.",
  "parameters": {
    "type": "object",
    "properties": {
      "name": {
        "type": "string",
        "description": "File name (without extension) under which the audio will be saved."
      },
      "content": {
        "type": "string",
        "description": "The story or text content that should be converted into spoken audio."
      }
    },
    "required": ["name", "content"]
  }
}

In [ ]:
play_story_json = {
  "name": "play_story",
  "description": "Play a previously saved WAV audio file from the local sample_data directory.",
  "parameters": {
    "type": "object",
    "properties": {
      "file_name": {
        "type": "string",
        "description": "Name of the audio file to play (without the .wav extension)."
      }
    },
    "required": ["file_name"]
  }
}

In [87]:
get_all_stories_json = {
  "name": "get_all_stories",
  "description": "Retrieve a list of all saved story narration audio files. This tool scans the local storage directory and returns the names of all .wav audio files generated from story narrations.",
  "parameters": {
    "type": "object",
    "properties": {},
    "required": []
  }
}

In [92]:
tools = [
    {"type": "function", "function": play_story_json},
    {"type": "function", "function": save_story_json},
    {"type": "function", "function": get_all_stories_json} # Use the JSON schema here
]

In [93]:
def handle_tool_calls(tool_calls):
    tool_outputs = []
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        if function_name == "save_story":
            output = save_story(**function_args)
            tool_outputs.append(output)
        elif function_name == "play_story":
            output = play_story(**function_args)
            tool_outputs.append(output)
        elif function_name == "get_all_stories":
            output = get_all_stories()
            tool_outputs.append(output)
    return tool_outputs


In [78]:
client  = OpenAI(api_key=gemini_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"

In [ ]:
print(client)

In [94]:
def chat(message, history):
    # Build initial messages
    messages = (
        [{"role": "system", "content": system_prompt}]
        + history
        + [{"role": "user", "content": message}]
    )

    done = False
    final_response = ""
    audio_to_play_path = None # Variable to store audio path if play_story is called

    while not done:
        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            tools=tools,
        )

        choice = response.choices[0]
        assistant_message = choice.message
        finish_reason = choice.finish_reason

        # Case 1: LLM wants to call tool(s)
        if finish_reason == "tool_calls":
            tool_calls = assistant_message.tool_calls

            # Add assistant tool-call message
            messages.append({
                "role": "assistant",
                "content": assistant_message.content,
                "tool_calls": tool_calls,
            })

            # Execute tools
            tool_results = handle_tool_calls(tool_calls)

            # Add tool results back to conversation and check for audio to play
            for tool_call, result in zip(tool_calls, tool_results):
                if tool_call.function.name == "play_story":
                    audio_to_play_path = result # Capture the audio path string
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": tool_call.function.name,
                    "content": str(result), # Still pass string representation to LLM
                })

        # Case 2: Final assistant response
        else:
            final_response = assistant_message.content or ""
            done = True

    # Return the final response to Gradio, including audio if available
    if audio_to_play_path:
        return final_response, audio_to_play_path
    else:
        return final_response

In [73]:
import gradio as gr

In [ ]:
import gradio as gr

# -------------------------------
# Wrapper for Gradio compatibility
# -------------------------------
def chat_wrapper(message, history):
    result = chat(message, history)

    # Gradio's chatbot with type='messages' expects a list of dictionaries
    new_history = history + [{'role': 'user', 'content': message}]

    # Case 1: text + audio
    if isinstance(result, tuple):
        text, audio_path = result
        new_history.append({'role': 'assistant', 'content': text})
        return new_history, audio_path

    # Case 2: text only
    new_history.append({'role': 'assistant', 'content': result})
    return new_history, None


# -------------------------------
# UI
# -------------------------------
with gr.Blocks(
    theme=gr.themes.Soft(),
    css="""
        .title { font-size: 2.3rem; font-weight: 700; }
        .subtitle { color: #666; }
    """
) as demo:

    gr.Markdown(
        """
        <div class="title">📖 Story Generator Agent</div>
        <div class="subtitle">
        Multi-tool LLM agent with narration support
        </div>
        """
    )

    gr.Markdown("...")

    chatbot = gr.Chatbot(
        label="🧠 Story Agent",
        height=420,
        type="messages", # Specify type as 'messages' for OpenAI-style dicts
        allow_tags=False # Explicitly set to silence deprecation warning
    )

    audio_output = gr.Audio(
        label="🔊 Story Narration",
        autoplay=True
    )

    with gr.Row():
        msg = gr.Textbox(
            placeholder="Write a story about a time-traveling detective...",
            scale=4,
            show_label=False
        )
        send = gr.Button("✨ Generate", scale=1)

    send.click(
        fn=chat_wrapper,
        inputs=[msg, chatbot],
        outputs=[chatbot, audio_output]
    )

    msg.submit(
        fn=chat_wrapper,
        inputs=[msg, chatbot],
        outputs=[chatbot, audio_output]
    )

demo.launch(
    debug=True
)